#### LeetCode - SQL - #1070

Refer [1070. Product Sales Analysis III](https://leetcode.com/problems/product-sales-analysis-iii/description/)

Write a solution to select the  **product id**,  **year**,  **quantity**, and  **price**  for the  **first year**  of every product sold.

Return the resulting table in  **any order**.

In [24]:
data_sales = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
columns_sales = ['sale_id', 'product_id', 'year', 'quantity', 'price']
schema_sales = {'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'}

data_product = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
columns_product = ['product_id', 'product_name']
schema_product = {'product_id':'Int64', 'product_name':'object'}

In [25]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

In [ ]:
# Create Spark Dataframe using Pandas
import pandas

pandas_df_sales = pandas.DataFrame(data=data_sales, columns=columns_sales).astype(schema_sales)
pandas_df_product = pandas.DataFrame(data=data_product, columns=columns_product).astype(schema_product)

df_sales = spark_context.createDataFrame(pandas_df_sales)
df_sales.printSchema()

df_product = spark_context.createDataFrame(pandas_df_product)
df_product.printSchema()

In [ ]:
# Create Spark Dataframe using PySpark functions - 1
schema_sales = 'sale_id integer, product_id integer, year integer, quantity integer, price integer'
schema_product = 'product_id integer, product_name string'

df_sales = spark_context.createDataFrame(data=data_sales, schema=schema_sales)
df_sales.printSchema()

df_product = spark_context.createDataFrame(data=data_product, schema=schema_product)
df_product.printSchema()

In [ ]:
# Create Spark Dataframe using PySpark functions - 2
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema_sales = StructType([
    StructField('sale_id', IntegerType(), False),
    StructField('product_id', IntegerType(), False),
    StructField('year', IntegerType(), False),
    StructField('quantity', IntegerType(), False),
    StructField('price', IntegerType(), False),
])
df_sales = spark_context.createDataFrame(data=data_sales, schema=schema_sales)
df_sales.printSchema()

schema_product = StructType([
    StructField('product_id', IntegerType(), False),
    StructField('product_name', StringType(), True)
])
df_product = spark_context.createDataFrame(data=data_product, schema=schema_product)
df_product.printSchema()

#### Using Join

In [ ]:
from pyspark.sql import functions as F

df_sales_grouped \
    = df_sales.groupBy(df_sales.product_id).agg(
        F.min(df_sales.year).alias('earliest_year')
    )
df_sales_grouped.show()

In [33]:
df_sales = df_sales.alias('sales')
df_sales_grouped = df_sales_grouped.alias('product_year')
df_product_alias = df_product.alias('product')

(df_sales_grouped.join(
    df_sales,
    ((F.col('product_year.product_id') == F.col('sales.product_id')) & (F.col('product_year.earliest_year') == F.col('sales.year'))),
    'inner'
).join(
    df_product_alias,
    (F.col('sales.product_id') == F.col('product.product_id')),
    'inner'
).select(
    F.col('sales.product_id'), F.col('product.product_name'), F.col('sales.year'), F.col('sales.quantity'), F.col('sales.price')
).show())

+----------+------------+----+--------+-----+
|product_id|product_name|year|quantity|price|
+----------+------------+----+--------+-----+
|       100|       Nokia|2008|      10| 5000|
|       200|       Apple|2011|      15| 9000|
+----------+------------+----+--------+-----+
